In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow import keras as K
from tfsnippet.modules import Sequential
from tfsnippet.utils import get_variables_as_dict, VariableSaver
from donut import complete_timestamp, standardize_kpi
from donut import Donut

# Import and pre-process data

In [2]:
# Read the raw data.
df = pd.read_csv("data/g.csv")
timestamp, values, labels = df["timestamp"],df["value"], df["label"]
# If there is no label, simply use all zeros.
labels = np.zeros_like(values, dtype=np.int32)

# Complete the timestamp, and obtain the missing point indicators.
timestamp, missing, (values, labels) = complete_timestamp(timestamp, (values, labels))

# Split the training and testing data.
test_portion = 0.25
test_n = int(len(values) * test_portion)
train_values, test_values = values[:-test_n], values[-test_n:]
train_labels, test_labels = labels[:-test_n], labels[-test_n:]
train_missing, test_missing = missing[:-test_n], missing[-test_n:]

# Standardize the training and testing data.
train_values, mean, std = standardize_kpi(
    train_values, excludes=np.logical_or(train_labels, train_missing))
test_values, _, _ = standardize_kpi(test_values, mean=mean, std=std)

# Create the model

In [3]:
with tf.variable_scope('model') as model_vs:
    model = Donut(
        h_for_p_x=Sequential([
            K.layers.Dense(100, kernel_regularizer=K.regularizers.l2(0.001),
                           activation=tf.nn.relu),
            K.layers.Dense(100, kernel_regularizer=K.regularizers.l2(0.001),
                           activation=tf.nn.relu),
        ]),
        h_for_q_z=Sequential([
            K.layers.Dense(100, kernel_regularizer=K.regularizers.l2(0.001),
                           activation=tf.nn.relu),
            K.layers.Dense(100, kernel_regularizer=K.regularizers.l2(0.001),
                           activation=tf.nn.relu),
        ]),
        x_dims=120,
        z_dims=5,
    )

# Training and Testing (+ Saving)

In [4]:
from donut import DonutTrainer, DonutPredictor

trainer = DonutTrainer(model=model, model_vs=model_vs)
predictor = DonutPredictor(model)

with tf.Session().as_default():
    trainer.fit(train_values, train_labels, train_missing, mean, std)
    test_score = predictor.get_score(test_values, test_missing)
    var_dict = get_variables_as_dict(model_vs)

    # save variables to `save_dir`
    save_dir = "models/"
    saver = VariableSaver(var_dict, save_dir)
    saver.save()

32.3666 (±6.89898); valid loss: 52.6482
[Epoch 226/256, Step 99500, ETA 47s] step time: 0.003152s (±0.01017s); valid time: 0.1023s; loss: -32.4286 (±7.57531); valid loss: 55.8698
[Epoch 226/256, Step 99600, ETA 46.66s] step time: 0.003148s (±0.01057s); valid time: 0.1062s; loss: -31.2287 (±8.26531); valid loss: 63.3405
[Epoch 226/256, Step 99700, ETA 46.31s] step time: 0.003184s (±0.01046s); valid time: 0.1052s; loss: -32.4575 (±7.00791); valid loss: 55.605
[Epoch 226/256, Step 99800, ETA 45.97s] step time: 0.003138s (±0.01031s); valid time: 0.1035s; loss: -31.7053 (±6.82148); valid loss: 57.4179
[Epoch 227/256, Step 99900, ETA 45.63s] step time: 0.003193s (±0.01034s); valid time: 0.1039s; loss: -31.0612 (±7.77678); valid loss: 54.7747
[Epoch 227/256, Step 100000, ETA 45.29s] step time: 0.003243s (±0.01051s); valid time: 0.1056s; loss: -31.9256 (±6.56529); valid loss: 52.6022
[Epoch 227/256, Step 100100, ETA 44.94s] step time: 0.003204s (±0.01066s); valid time: 0.1071s; loss: -32.0013 

# Restore the model

In [5]:
with tf.Session().as_default():
    # Restore variables from `save_dir`.
    saver = VariableSaver(get_variables_as_dict(model_vs), save_dir)
    saver.restore()

INFO:tensorflow:Restoring parameters from /home/samsepiol/Desktop/ANM project/donut/temp/models/variables.dat
